In [4]:
import torch
from safetensors.torch import load_file
from transformers import AutoModelForSequenceClassification

# Load the pretrained model
pretrained_model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

# Paths to checkpoints
pretrained_checkpoint = "C:/Users/Anneth/TST/pretrained_albert/model.safetensors"
finetuned_checkpoint = "C:/Users/Anneth/TST/fine_tuned_albert/model.safetensors"

class TaskVector():
    def __init__(self, pretrained_checkpoint=None, finetuned_checkpoint=None, vector=None):
        """Initializes the task vector from a pretrained and a finetuned checkpoints.
        
        This can either be done by passing two state dicts (one corresponding to the
        pretrained model, and another to the finetuned model), or by directly passying in
        the task vector state dict.
        """
        if vector is not None:
            self.vector = vector
        else:
            assert pretrained_checkpoint is not None and finetuned_checkpoint is not None
            with torch.no_grad():
                #if pretrained_checkpoint.endswith(".safetensors"):
                pretrained_state_dict = load_file(pretrained_checkpoint)
                #else:
                #    pretrained_state_dict = torch.load(pretrained_checkpoint).state_dict()
            
                #if finetuned_checkpoint.endswith(".safetensors"):
                finetuned_state_dict = load_file(finetuned_checkpoint)
                #else:
                #    finetuned_state_dict = torch.load(finetuned_checkpoint).state_dict()
                
                self.vector = {}
                for key in pretrained_state_dict:
                    if pretrained_state_dict[key].dtype in [torch.int64, torch.uint8]:
                        continue
                    self.vector[key] = finetuned_state_dict[key] - pretrained_state_dict[key]
    
    def __add__(self, other):
        """Add two task vectors together."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                if key not in other.vector:
                    print(f'Warning, key {key} is not present in both task vectors.')
                    continue
                new_vector[key] = self.vector[key] + other.vector[key]
        return TaskVector(vector=new_vector)

    def __radd__(self, other):
        if other is None or isinstance(other, int):
            return self
        return self.__add__(other)

    def __neg__(self):
        """Negate a task vector."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                new_vector[key] = - self.vector[key]
        return TaskVector(vector=new_vector)

    def apply_to(self, pretrained_model, scaling_coef=1.0): # changed "pretrained_checkpoint" to "pretrained_model"
        """Apply a task vector to a pretrained model."""
        with torch.no_grad():
            #if pretrained_checkpoint.endswith(".safetensors"):
            #    pretrained_state_dict = load_file(pretrained_checkpoint)
            #else:
            pretrained_state_dict = pretrained_model.state_dict()
            #    pretrained_state_dict = torch.load(pretrained_checkpoint).state_dict()
            new_state_dict = {}
            #pretrained_state_dict = pretrained_model.state_dict()
            for key in pretrained_state_dict:
                if key not in self.vector:
                    print(f'Warning: key {key} is present in the pretrained state dict but not in the task vector')
                    continue
                new_state_dict[key] = pretrained_state_dict[key] + scaling_coef * self.vector[key]
        pretrained_model.load_state_dict(new_state_dict, strict=False)
        return pretrained_model



Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
